In [1]:
# system
import os
import sys
import logging
import pickle
import multiprocessing as mp
from functools import partial
from collections import Counter

# externals 
import uproot
import torch
import pandas as pd
import awkward as ak
import numpy as np
from torch_geometric.data import DataLoader, Data
from matplotlib import pyplot as plt
import particle 
from matplotlib import colors
from matplotlib.patches import Ellipse
from particle import PDGID
from particle import Particle 
from particle.pdgid import is_meson
import mplhep as hep
import scipy.stats
hep.style.use("CMS")

# custom modules
sys.path.append('../graph_construction')
from build_tau_graphs import *

# setup logging
log_format = '%(asctime)s %(levelname)s %(message)s'
logging.basicConfig(level=logging.INFO, format=log_format)
logging.info('Initializing')

2022-01-18 14:18:07,511 INFO Initializing


In [2]:
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 given in range [-pi,pi]"""
    dphi = phi2 - phi1
    if (dphi > np.pi): dphi -= 2*np.pi
    if (dphi < -np.pi): dphi += 2*np.pi
    #dphi[dphi > np.pi] -= 2*np.pi
    #dphi[dphi < -np.pi] += 2*np.pi
    return dphi

def calc_dR(eta1, phi1, eta2, phi2):
    return np.sqrt((eta1 - eta2)**2
                   + calc_dphi(phi1, phi2)**2)

def match_PF_jets(taus, jets, vis):
    dR_mins, vxy, v = [], [], []
    pt_jet, pt_tau, pt_vis =  [], [], []
    jet_spacing = []
    
    # loop over all events 
    for i in range(len(taus['pt'])):
        dR_min, j_min = 100, 0
        dR_jet_min, j_jet_min = 100, 0
        
        # loop over all PF jets
        njets = len(jets['pt'][i])
        if (njets < 2): continue
        jet_idx = np.random.choice(njets, 1)[0]
        for j in range(len(jets['pt'][i])):
            dR = calc_dR(jets['eta'][i,j], jets['phi'][i,j],
                         taus['eta'][i], taus['phi'][i])
            dR_jet = calc_dR(jets['eta'][i,j], jets['phi'][i,j],
                             jets['eta'][i,jet_idx], jets['phi'][i,jet_idx])
            
            # store closest jet
            if dR<dR_min:
                dR_min, j_min = dR, j
            if ((dR_jet<dR_jet_min) and (j!=jet_idx)):
                dR_jet_min, j_jet_min = dR_jet, j
                      
        # otherwise store variables of interest
        print(i)
        dR_mins.append(dR_min)
        pt_tau.append(taus['pt'][i])
        pt_jet.append(jets['pt'][i,j_min])
        pt_vis.append(ak.sum(vis['pt'][i]))
        vxy.append(taus['vxy'][i])
        v.append(np.sqrt(taus['vxy'][i]**2 + taus['vz'][i]**2))
        jet_spacing.append(dR_jet_min)
    
    return pd.DataFrame({'dR_min': dR_mins,
                         'vxy': vxy, 'v': v,
                         'pt_jet': pt_jet,
                         'pt_tau': pt_tau,
                         'pt_vis': pt_vis,
                         'jet_spacing': jet_spacing})

In [ ]:
skim_dir = '../skims'
skims = os.listdir(skim_dir)
out = {}
for skim in skims:
    decay_mode = skim.split('.')[0]
    f = os.path.join(skim_dir, skim)
    with open(f, 'rb') as handle:
        data = pickle.load(handle)
        logging.info(f"Loaded {f}")
        
    args = {'tau_pt_min': 5, 'ecal_min': 0.05, 'hcal_min': 0.05}
    gen = filter_gen_arrays(data['gen_arrays'], args=args)
    reco = filter_reco_arrays(data['reco_arrays'], args=args)
    rec = reco['rec_hits']  
    jets = reco['jets']
    tau = gen['tau']
    vis = gen['vis']
    inv = gen['inv']
    out[decay_mode] = match_PF_jets(tau, jets, vis)
    logging.info(f"Storing output data: {out[decay_mode]}")

In [3]:
skim_dir = '../skims'
skims = os.listdir(skim_dir)
out = {}
for skim in skims:
    decay_mode = skim.split('.')[0]
    f = os.path.join(skim_dir, skim)
    with open(f, 'rb') as handle:
        data = pickle.load(handle)
        logging.info(f"Loaded {f}")
        
    args = {'tau_pt_min': 5, 'ecal_min': 0.05, 'hcal_min': 0.05}
    gen = filter_gen_arrays(data['gen_arrays'], args=args)
    reco = filter_reco_arrays(data['reco_arrays'], args=args)
    rec = reco['rec_hits']  
    jets = reco['jets']
    tau = gen['tau']
    vis = gen['vis']
    inv = gen['inv']

2022-01-18 14:18:37,355 INFO Loaded ../skims/ThreeProngsNoPi0.p
2022-01-18 14:19:07,367 INFO Loaded ../skims/ThreeProngsOnePi0.p


KeyboardInterrupt: 

In [11]:
ieta = np.array(reco['rec_hits']['eb']['ieta'][0])
iphi = np.array(reco['rec_hits']['eb']['iphi'][0])
ixy = np.column_stack((ieta, iphi))
print(len(ixy), len(np.unique(ixy, axis=0)))


364 364


In [ ]:
import seaborn as sns

# plot the dR(matched jet, tau) distribution
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
plt.figure(dpi=200)
#sns.histplot(x=all_dm_out['dR_min'], element='step', label='all dms')
for name, data in out.items():
    sns.histplot(x=data['dR_min'], element='step', fill=False, label=name)
plt.xlabel(r'Minimum $dR(j,\tau_h)$')
plt.legend(loc='best')
plt.xlim([0,3])
plt.show()

In [ ]:
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
plt.figure(dpi=200)
print(all_dm_out['jet_spacing'])
sns.histplot(x=all_dm_out['jet_spacing'], element='step', label='all dms')
#for name, data in out.items():
#    sns.histplot(x=data['dR_min'], element='step', fill=False, label=name)
plt.xlabel(r'Avg Jet Spacing')
plt.legend(loc='best')
plt.xlim([0,3])
plt.show()

In [ ]:
# plot dR vs. vxy
plt.figure(dpi=200)
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
print(np.min(all_dm_out['vxy']))
sns.regplot(x=all_dm_out['vxy'], y=all_dm_out['dR_min'], x_bins=np.arange(0,100,5), 
            marker='o', color='blue', fit_reg=False)
plt.xlabel('$v_{xy}$ [cm]')
plt.ylabel(r'$dR(j,\tau_h)$')
plt.ylim([0.3, 0.6])
plt.show()

In [ ]:
# plot dR vs. vxy
plt.figure(dpi=200)
for name, data in out.items():
    sns.regplot(x=data['vxy'], y=data['dR_min'], x_bins=5, marker='o',
                fit_reg=False, label=name, scatter_kws={})
plt.legend(loc='best')
plt.xlabel('$v_{xy}$ [cm]')
plt.ylabel(r'$dR(j,\tau_h)$')
plt.show()

In [ ]:
# plot the dR(matched jet, tau) distribution
fig = plt.figure(dpi=200)
for name, data in out.items():
    dRs = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]
    match = []
    no_match = []
    for dR in dRs:
        matched = (data['dR_min'] < dR)
        match.append(ak.sum(matched)/len(matched))
        
    sns.scatterplot(x=dRs, y=match, label=name)   
    
plt.legend(loc='best')
plt.xlabel('dR Match Threshold')
plt.ylabel('Fraction of Matches')
plt.show()

In [ ]:
# plot vis pt vs jet pt for matched and not matched
plt.figure(dpi=200)
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
matched = (all_dm_out['dR_min'] < 0.7)
sns.regplot(x=all_dm_out['pt_vis'][matched], y=all_dm_out['pt_jet'][matched], 
            marker='o', x_bins=np.arange(0,100,2),
            color='green', fit_reg=True, label=r'$dR(j,\tau_h)<0.7$')
sns.regplot(x=all_dm_out['pt_vis'][~matched], y=all_dm_out['pt_jet'][~matched], 
            marker='o', x_bins=np.arange(0,100,2),
            color='blue', fit_reg=True, label=r'$dR(j,\tau_h)>0.7$')
plt.xlabel('$p_T^\mathrm{vis}$ [GeV]')
plt.ylabel('$p_T^\mathrm{jet}$ [GeV]')
plt.xlim([0,100])
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(dpi=200)
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
matched = (all_dm_out['dR_min'] < 0.7)
sns.regplot(x=all_dm_out['vxy'], y=matched, x_bins=np.arange(0,100,5),
            marker='o', fit_reg=False)
plt.xlabel('$v_{xy}$ [cm]')
plt.ylabel(r'Fraction with $dR(j,\tau_h)<0.7$')
plt.show()

In [ ]:
# plot the dR(matched jet, tau) distribution
for i, (name, data) in enumerate(out.items()):
    fig = plt.figure(dpi=200)
    dRs = [0.15, 0.3, 0.4, 0.5]
    for dR in dRs:
        matched = (data['dR_min'] < dR)
        sns.regplot(x=data['vxy'], y=matched, x_bins=np.arange(0,100,10), 
                    marker='o', fit_reg=False, label=f"dR<{dR}")
    plt.title(name)
    plt.legend(loc='best')
    plt.xlabel('$v_{xy}$ [cm]')
    plt.ylabel(r'Fraction with $dR(j,\tau_h)<0.4$')
    plt.show()

In [ ]:
plt.figure(dpi=200)
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
matched = (all_dm_out['dR_min'] < 0.4)

sns.regplot(x=all_dm_out['vxy'], y=matched, x_bins=np.arange(0,100,5),
            marker='o', fit_reg=False)
plt.xlabel('$v_{xy}$ [cm]')
plt.ylabel(r'Fraction with $dR(j,\tau_h)<0.4$')
plt.show()

In [ ]:
plt.figure(dpi=200)
all_dms = [val for val in out.values()]
all_dm_out = pd.concat(all_dms).reset_index()
matched = (all_dm_out['dR_min'] < 0.4)
y = (all_dm_out['pt_jet']-all_dm_out['pt_vis'])/all_dm_out['pt_vis']
sns.regplot(x=all_dm_out['vxy'][matched], y=y[matched], x_bins=np.arange(0,100,5),
            marker='o', fit_reg=False, label=r"$dR(j,\tau_h)<0.4$")
sns.regplot(x=all_dm_out['vxy'][~matched], y=y[~matched], x_bins=np.arange(0,100,5),
            marker='o', fit_reg=False, label=r"$dR(j,\tau_h)>0.4$")
plt.xlabel('$v_{xy}$ [cm]')
plt.ylabel(r'$(p_T^\mathrm{vis}-p_T^\mathrm{jet})/p_T^\mathrm{vis}$')
plt.legend(loc='best')
plt.show()